In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import os


In [2]:
# Read in the data
train = pd.read_csv('data/train.csv')
train_labels = pd.read_csv('data/train_labels.csv')

In [3]:
# setting copy warning to false
pd.options.mode.chained_assignment = None

In [4]:
# calculate f1 score of predicting always 1

y_true = train_labels['correct']
y_pred = np.ones(len(y_true))

from sklearn.metrics import f1_score
f1_score(y_true, y_pred, average='macro') # very important to turn on average='macro' to get the right score

0.41370801521177464

In [5]:
# keep only level_group 0-4
train = train[train['level_group'] == '0-4']

In [6]:
train.sample(5)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
13105499,21040212461167524,15,34435,navigate_click,undefined,0,NaN,-323.023056,-181.928160,92.0,480.0,NaN,NaN,NaN,tunic.historicalsociety.closet,NaN,1,1,1,0-4
17300763,21080310481252784,100,122888,navigate_click,undefined,3,NaN,615.298758,-140.835283,824.0,425.0,NaN,NaN,toentry,tunic.historicalsociety.collection,NaN,0,0,1,0-4
7378737,21010415225372040,21,34097,object_click,basic,0,NaN,-207.484644,-193.883252,742.0,497.0,NaN,NaN,retirement_letter,tunic.historicalsociety.closet,NaN,0,0,1,0-4
19689642,21100513124781364,35,100969,navigate_click,undefined,1,NaN,-24.759651,-3.879226,209.0,332.0,NaN,NaN,groupconvo,tunic.historicalsociety.entry,NaN,1,0,1,0-4
5263673,21000509011083970,103,268452,object_click,basic,3,NaN,184.371898,301.675948,544.0,120.0,NaN,NaN,tunic.hub.slip,tunic.historicalsociety.collection,NaN,0,0,1,0-4


In [7]:
train.drop(['screen_coor_x', 'screen_coor_y', 'level_group', 'hq', 'music', 'text', 'fqid', ], axis=1, inplace=True)

In [8]:
# Get max and min of room_coor_x and room_coor_y for each room_fqid
room_coor_x_max = train.groupby('room_fqid')['room_coor_x'].max()
room_coor_x_min = train.groupby('room_fqid')['room_coor_x'].min()

room_coor_y_max = train.groupby('room_fqid')['room_coor_y'].max()
room_coor_y_min = train.groupby('room_fqid')['room_coor_y'].min()

# join the results into a dataframe
room_coor_x = pd.concat([room_coor_x_max, room_coor_x_min], axis=1)
room_coor_x.columns = ['room_coor_x_max', 'room_coor_x_min']

room_coor_y = pd.concat([room_coor_y_max, room_coor_y_min], axis=1)
room_coor_y.columns = ['room_coor_y_max', 'room_coor_y_min']

# join the results into a dataframe
room_coor = pd.concat([room_coor_x, room_coor_y], axis=1)

room_coor # These coordinates will dictate the min max axis values in each plot


,room_coor_x_max,room_coor_x_min,room_coor_y_max,room_coor_y_min
room_fqid,,,,
tunic.capitol_0.hall,937.826685,-870.331689,462.335042,-559.484403
tunic.historicalsociety.basement,764.564032,-747.335082,447.069000,-484.499636
tunic.historicalsociety.closet,925.801770,-1063.881885,332.872127,-393.703169
tunic.historicalsociety.collection,799.918926,-775.811279,474.637586,-493.592464
tunic.historicalsociety.entry,1172.531821,-1218.142341,543.616424,-591.735917
tunic.historicalsociety.stacks,508.797338,-511.667195,316.595477,-347.434545
tunic.kohlcenter.halloffame,991.377507,-1001.164257,330.000000,-330.000000


In [9]:
train['name'].value_counts()

undefined    1926175
basic        1862968
close         157166
open           33179
prev            1007
next             510
Name: name, dtype: int64

In [10]:
train['event_name'].value_counts() # Seems more informative than name, will use as marker

navigate_click        1807806
cutscene_click         787584
person_click           484889
object_click           364862
notification_click     183243
object_hover           107127
notebook_click          81733
map_click               54068
map_hover               45130
observation_click       40850
checkpoint              23713
Name: event_name, dtype: int64

In [11]:
# Check how many rooms each level has
train.groupby('level')['room_fqid'].agg('nunique')

level
0    1
1    3
2    5
3    6
4    2
Name: room_fqid, dtype: int64

In [12]:
# mean number of events for each room per session'
train.groupby(['session_id', 'room_fqid'])['event_name'].count().groupby('room_fqid').mean()
# Only create plots for rooms with alot of events

room_fqid
tunic.capitol_0.hall                   3.357058
tunic.historicalsociety.basement       4.540531
tunic.historicalsociety.closet        36.383711
tunic.historicalsociety.collection    37.334196
tunic.historicalsociety.entry         45.391690
tunic.historicalsociety.stacks         5.087227
tunic.kohlcenter.halloffame           39.873907
Name: event_name, dtype: float64

# Important:

We use data from level_groups 0-4 for questions 1-3 only.

More info:
https://www.kaggle.com/code/cdeotte/xgboost-baseline-0-680

In [13]:
train_labels['question'] = train_labels['session_id'].apply(lambda x: x.split('_')[1])
train_labels['session_id'] = train_labels['session_id'].apply(lambda x: x.split('_')[0])
train_labels = train_labels[train_labels['question'] == 'q1']

In [14]:
# Let's start for question 1
train_labels = train_labels[train_labels['question'] == 'q1']

In [15]:
train_labels['correct'].mean() # 73% of the time the user got the question correct

0.7274849333672863

In [16]:
from sklearn.preprocessing import LabelEncoder

# encode event_name
le = LabelEncoder()
le.fit(train['event_name'])
train['event_name'] = le.transform(train['event_name'])

# create array of 40 different quantiles for elapsed_time
#quantiles = np.linspace(0, 1, 40)

# get quantiles of elapsed_time
#quantiles_time = room['elapsed_time'].quantile(quantiles)
# this will be used later on. Now let's focus on the room coordinates



# Feature engineering:

In [17]:
rooms = train['room_fqid'].unique()        

In [18]:
len(rooms)

7

In [19]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [20]:
def plot_room(user_id, room_id, max_x, max_y, min_x, min_y, ax):
    user = train[train['session_id'] == user_id]
    # drop rows with nan values in room_coor_x and room_coor_y
    user = user.dropna(subset=['room_coor_x', 'room_coor_y'])
    # fillna of event_name with 'unknown'
    user['event_name'] = user['event_name'].fillna('unknown')

    # sort user by elapsed_time
    user = user.sort_values('elapsed_time')
    # set index to follow the order of elapsed_time
    user = user.reset_index(drop=True)
    user['index'] = user.index

    room = user[user['room_fqid'] == room_id]


    # if index increases by more than 5, then it's a new index_group
    room['index_group'] = room['index'] - room['index'].shift(1)
    room['index_group'] = room['index_group'].apply(lambda x: 1 if x > 5 else 0)
    room['index_group'] = room['index_group'].cumsum()

    room['elapsed_time'] = room['elapsed_time'] - room['elapsed_time'].shift(1)
    room['elapsed_time'] = room['elapsed_time'].fillna(0)

    


    # if index_group > 2 then it is 3
    room['index_group'] = room['index_group'].apply(lambda x: 3 if x > 2 else x)

    grouped = room.groupby('index_group')

    # Define a list of colors for the different index_group values
    colors = ['red', 'blue', 'green', 'yellow']

    # Iterate over the groups and plot each one with a different color
    for i, (name, group) in enumerate(grouped):
        color = colors[i]
        ax.scatter(group['room_coor_x'], group['room_coor_y'], c=group['event_name'], label=name, cmap='viridis', vmin=0, vmax=10)
        ax.plot(group['room_coor_x'], group['room_coor_y'], color=color)

    # Add a legend and title
    handles, labels = ax.get_legend_handles_labels()

    # Remove borders and axis
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)

    return ax


In [21]:
def plot_rooms(session_id):
    user = train[train['session_id'] == session_id]

    fig, ax = plt.subplots(2, 4, figsize=(16, 9))
    axes = ax.flatten()

    for i, room_id in enumerate(rooms):
        max_x, min_x, max_y, min_y = room_coor.loc[room_id].values
        plot_room(session_id, room_id, max_x, max_y, min_x, min_y, axes[i])

    # Remove borders and axis from all axes
    for ax in axes:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])

    # Adjust spacing between subplots
    plt.subplots_adjust(wspace=0.1, hspace=0.3)

    return axes



In [22]:
# convert to dataframe
processed_data = os.listdir('user_data')
processed_data = pd.DataFrame(processed_data, columns=['session_id'])
processed_data['session_id'] = processed_data['session_id'].apply(lambda x: x.split('.')[0])
processed_data['session_id'] = processed_data['session_id'].astype('int64')

# get unprocessed data
unprocessed_data = train[~train['session_id'].isin(processed_data['session_id'])]

failed_sessions = []
success = 0
fails = 0
total_sessions = len(unprocessed_data['session_id'].unique())
for i, session_id in enumerate(unprocessed_data['session_id'].unique()):
    try:
        axes = plot_rooms(session_id)
        plt.savefig(f'user_data/{session_id}.png', dpi=50)
        plt.close()
        success += 1
    except Exception as e:
        try:
            print(e)
            plt.close()
        except:
            pass
        failed_sessions.append(session_id)
        fails += 1
    
    success_rate = success / (success + fails) * 100 if success + fails > 0 else 0
    progress_bar = '[' + '#' * int((i + 1) / total_sessions * 20) + ' ' * (20 - int((i + 1) / total_sessions * 20)) + ']'
    print(f'Processed {success}/{total_sessions} sessions, failed {fails} sessions, success rate {success_rate:.2f}% \t {progress_bar}', end='\r')